<a href="https://colab.research.google.com/github/ddekun/PyTorch_framework/blob/Lesson9/lesson9/hw9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Фреймворк PyTorch для разработки искусственных нейронных сетей

### Урок 9. Трансформер

Домашнее задание
Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
Сделайте предсказания на всем df_val. Посчитайте метрику качества.
Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.9 MB/s eta 0:00:00


In [3]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 6.4 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

from transformers import pipeline
from tqdm import tqdm
from torchmetrics import F1Score
from transformers import BertTokenizer, BertForSequenceClassification

In [5]:
model = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny-toxicity')
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29564, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, e

In [6]:
sentiment = pipeline("text-classification", model='cointegrated/rubert-tiny-toxicity')
sentiment(["Этот ресторан отличный", 'А тот ресторан - полная жопа'])

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'non-toxic', 'score': 0.9993882179260254},
 {'label': 'insult', 'score': 0.9857767224311829}]

In [7]:
tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny-toxicity')

example_text = 'Пример текста для токенизации'
bert_input = tokenizer(example_text, padding='max_length', max_length=10,
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])

tensor([[    2,  3086, 10885, 22723,   871, 24302,  3464, 10880,     3,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])


In [8]:
example_text = tokenizer.decode(bert_input.input_ids[0])
print(example_text)

[CLS] Пример текста для токенизации [SEP] [PAD]


In [9]:
from google.colab import drive
drive.mount('/content/drive')

!unzip /content/drive/MyDrive/Geekbrains/PyTorch_framework/Lesson9/data.zip

df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

df_train.shape, df_val.shape

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/Geekbrains/PyTorch_framework/Lesson9/data.zip
  inflating: train.csv               
  inflating: val.csv                 


((181467, 3), (22683, 3))

In [10]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [11]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

In [12]:
idx = 35
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
sentiment(df_train.iloc[idx]['text'])

@KravchenkoNasty да #echelonlovesfathers Ну у меня одна подруга слушает рок, ахах, я её прям люблю :D #echelonlovesfathers
label is 1


[{'label': 'non-toxic', 'score': 0.9904237389564514}]

In [13]:
df_train['text'] = df_train['text'].apply(lambda x: x.lower())
df_val['text'] = df_val['text'].apply(lambda x: x.lower())

In [14]:
class TwitterDataset(torch.utils.data.Dataset):

    def __init__(self, txts, labels):
        self._labels = labels

        self.tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny-toxicity')
               #для каждого text возвращает батч с полями:
               #'inputs_ids' -- тензор размера (B,1,max_len) из id токенов
               #'token_type_ids' -- тензор размера (B,1,max_len) из id типов токенов
               #'attention_mask' -- тензор размера (B,1,max_len) из индексов, указывающих, на какие токеты модель должна обратить внима
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]

    def __len__(self):
        return len(self._txts)

    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [15]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [16]:
for txt, lbl in train_loader:
    print(txt.keys()) #словарь с ключами 'input_ids', 'token_type_ids', 'attention_mask'
    print(txt['input_ids'].shape) #тензор размера (B,1,max_len) из id токенов
    print(txt['attention_mask'].shape) #тензор размера (B,1,max_len) из индексов, указывающих, на какие токеты модель должна обратить внимание
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([64, 1, 10])
torch.Size([64, 1, 10])


In [17]:
class BertForSequenceClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.pretrained_model = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny-toxicity')
        self.dropout = nn.Dropout(dropout)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        pooled_output = self.pretrained_model(input_ids=x, attention_mask=mask, return_dict=False)[0]  #(B, 2)
        dropout_output = self.dropout(pooled_output)
        out = self.sigm(dropout_output)
        return out

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [19]:
model = BertForSequenceClassifier().to(device)
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.pretrained_model.classifier.parameters()]))

BertForSequenceClassifier(
  (pretrained_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(29564, 312, padding_idx=0)
        (position_embeddings): Embedding(512, 312)
        (token_type_embeddings): Embedding(2, 312)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-2): 3 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=312, out_features=312, bias=True)
                (key): Linear(in_features=312, out_features=312, bias=True)
                (value): Linear(in_features=312, out_features=312, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=31

In [21]:
from sklearn.metrics import f1_score


model.eval()

total_acc_val = 0
total_predictions = []
total_labels = []

for val_input, val_label in valid_loader:

    val_label = val_label.to(device)
    mask = val_input['attention_mask'].to(device)
    input_id = val_input['input_ids'].squeeze().to(device)
    output = model(input_id, mask)

    predictions = output.argmax(dim=1).cpu().numpy()
    total_predictions.extend(predictions)

    labels = val_label.cpu().numpy()
    total_labels.extend(labels)

    acc = (predictions == labels).sum().item()
    total_acc_val += acc

f1_val = f1_score(total_labels, total_predictions, average='weighted')

print(f'Val accuracy: {total_acc_val / len(valid_dataset):.3f}, F1 score: {f1_val:.3f}')


Val accuracy: 0.471, F1 score: 0.329


In [22]:
#компиляция модели
criterion = nn.CrossEntropyLoss()

# оптимизатор для последнего слоя модели
optimizer = Adam(model.pretrained_model.parameters(), lr=0.0001)

In [28]:
batch_size = 27
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
epochs = 5

for epoch_num in range(epochs):
    total_acc_train = 0
    total_loss_train = 0
    f1_train_values = []

    model.train()
    for train_input, train_label in tqdm(train_loader):

        output = model(input_id, mask)

        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()

        y_pred = output.argmax(dim=1).cpu().numpy()
        y_true = train_label.cpu().numpy()

        f1_train_value = f1_score(y_true, y_pred, average='micro') # You can choose 'macro' or 'weighted' if needed
        f1_train_values.append(f1_train_value)

        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()

    f1_train_avg = sum(f1_train_values) / len(f1_train_values)


In [ ]:
# считаю метрику дообученной модели на валидационном датасете

valid_f1 = F1Score().to(device)
model.eval()

total_acc_val = 0

for val_input, val_label in valid_loader:

    val_label = val_label.to(device)
    mask = val_input['attention_mask'].to(device)
    input_id = val_input['input_ids'].squeeze().to(device)
    output = model(input_id, mask)
    valid_f1(output, val_label)
    acc = (output.argmax(dim=1) == val_label).sum().item()
    total_acc_val += acc

print(f'Val accuracy: {total_acc_val/len(valid_dataset):.3f}, F1 score: {valid_f1.compute().item():.3f}')